In [8]:
include("ChebyshevPSI2025.jl")
using .ChebyshevPSI2025

In [9]:
using Interpolations
using CairoMakie
# using FastGaussQuadrature
# using SixelTerm

In [10]:
function f(x)
    return sin(10*x)
end


f (generic function with 1 method)

In [11]:
# coefficients = Vector{Float64}()
# cheb_vec= Vector{Function}()


# for i in 1:10
#     push!(cheb_vec,x->cos(i * acos(x)))
#     push!(coefficients,chebdot(f,cheb_vec[i]))
#     println("C_",i,"  ",coefficients[i])
# end

In [12]:
# xs = range(-1, 1, 1000)

# cheb_approx=Vector{Float64}()
# for i in 1:length(xs)
#     cheb_exp=0.
#     for j in 1:10
#         cheb_exp+=coefficients[j]*cheb(j,xs[i])
#     end
#     push!(cheb_approx,cheb_exp)
# end

In [65]:
function chebyshev_coefficient(f,number)
    coefficients = Vector{Float64}()
    cheb_vec= Vector{Function}()
    for i in 1:number
        if i == 1
            push!(coefficients,chebdot(f,x->1)/pi)
        else
            # push!(cheb_vec,x->cos((i-1) * acos(x)))
            push!(coefficients,chebdot(f,x->cos((i-1) * acos(x)))*2/pi)
        end
    end
    return coefficients
end

chebyshev_coefficient (generic function with 1 method)

In [64]:
chebdot(f,x->1)/pi

-1.0380977085332311e-16

In [71]:
function chebyshev_approx(f,number,xs)
    # xs = range(-1, 1, 1000)
    cheb_approx=Vector{Float64}()
    coefficients = chebyshev_coefficient(f,number)
    for i in 1:length(xs)
        cheb_exp=0.
        for j in 1:number
            cheb_exp+=coefficients[j]*cheb(j-1,xs[i])
        end
        push!(cheb_approx,cheb_exp)
    end
    return cheb_approx
end

chebyshev_approx (generic function with 1 method)

In [72]:
println(chebyshev_coefficient(f,10))

[-1.0380977085332311e-16, 0.08694549233772315, -1.2589695614126422e-16, -0.11675875861037355, 3.533949646070574e-17, -0.46812305637358725, 1.3252311172764653e-17, -0.43342183537010287, -3.754821498949985e-17, 0.5837113705302404]


In [73]:
xs = range(-1, 1, 1000)
cheb_approx=chebyshev_approx(f,10,xs);

In [74]:
figure = Figure()
ax = Axis(figure[1,1], title="Chebyshev expansion of f(x)=sin(10*x), with the number of polynomials n=10", xlabel="x", ylabel="y")
lines!(xs,f.(xs))
lines!(xs,cheb_approx)
#lines!(xs,f.(xs)-cheb_approx)
# figure
Axis(figure[2,1], title="Error in expansion", xlabel="x", ylabel="delta")
lines!(xs, f.(xs)- cheb_approx)
# figure
save("Chebyshev_10.png", figure)


CairoMakie.Screen{IMAGE}


In [75]:
cheb_approx_50=chebyshev_approx(f, 50,xs)
figure = Figure()
ax = Axis(figure[1,1], title="Chebyshev expansion of f(x)=sin(10*x), with the number of polynomials n=50", xlabel="x", ylabel="y")
lines!(xs,f.(xs))
lines!(xs,cheb_approx_50)
#lines!(xs,f.(xs)-cheb_approx)
# figure
Axis(figure[2,1], title="Error in expansion", xlabel="x", ylabel="delta")
lines!(xs, f.(xs)- cheb_approx_50)
# figure
save("Chebyshev_50.png", figure)

CairoMakie.Screen{IMAGE}


In [76]:
# Still using f(x)=sin(10*x) as an example, see when the Error is below 10^(-8)
n = n_final= 10
xs = range(-1, 1, 1000)
for n in 10:50
    cheb_approx=chebyshev_approx(f,n,xs)
    label = 1
    for i in 1:length(xs)
        err = abs(f(xs[i])-cheb_approx[i])
        if err>10^(-8)
            break
        end
        label+=1
    end
    if label==length(xs)+1
        n_final = n
        break
    else
        n+=1
    end
end
println("n_f ",n_final)

n_f 26


Q3:

To simplify the given expression and find the relation ( $c^i = D^i_j b^j$ ), we proceed step by step:

---

### The given expression for ( $c^i$ )
The Chebyshev expansion coefficient ( c^i ) is calculated using:
$[
c^i = \frac{1}{\kappa} \int_{-1}^1 f(x) T_i(x) \frac{1}{\sqrt{1-x^2}} dx,
]$
where $ \kappa = \pi/2$ for $ i > 0 $ and $ \kappa = \pi $ for $ i = 0 $.

Given that $ f(x) = g'(x) $ with $ g(x) = \sum_{j=0}^\infty b^j T_j(x) $, this becomes:
$[
c^i = \frac{1}{\kappa_i} \int_{-1}^1 g'(x) T_i(x) \frac{1}{\sqrt{1-x^2}} dx.
]$

$[
c^i = \frac{1}{\kappa_i} \sum_{j=0}^n b_j\int_{-1}^1 T'_j(x) T_i(x) \frac{1}{\sqrt{1-x^2}} dx.
]$

### T'(x)

$T'_{2n}(x)=2n*2*\sum_{k=1}^n T_{2k-1}(x)$

$T'_{2n+1}(x)=(2n+1)*[T_0(x)+2\sum_{k=1}^n T_{2k}(x)]$

In [106]:
coefficients = chebyshev_coefficient(f,100)
c_der=zeros(10)

for i in 1:10
   for j in 1:100
    if i == 1
        if (j-1)%2 ==0
            c_der[i]+=0
        else
            c_der[i]+=(j-1)*coefficients[j]
        end
    elseif (i-1)%2==0
        if (j-1)%2 ==0
            c_der[i]+=0
        elseif j-1<i
            c_der[i]+=0
        else
            c_der[i]+=2*(j-1)*coefficients[j]
        end
    else
        if (j-1)%2!=0
            c_der[i]+=0
        elseif j-1<i
            c_der[i]+=0
        else
            c_der[i]+=2*(j-1)*coefficients[j]
        end
    end
end
end

In [108]:
cheb_approx_der_1=Vector{Float64}()
for i in 1:length(xs)
    cheb_exp=0.
    for j in 1:10
        cheb_exp+=c_der[j]*cheb(j-1,xs[i])
    end
    push!(cheb_approx_der_1,cheb_exp)
end

In [82]:
function f_der(x)
    return 10*cos(10*x)
end

f_der (generic function with 1 method)

In [83]:
cheb_approx_der_2=chebyshev_approx(f_der,10,xs);

In [116]:
figure = Figure()
ax = Axis(figure[1,1], title="Chebyshev expansion of f'(x)=10*cos(10*x), n=10", xlabel="x", ylabel="y")
lines!(xs,f_der.(xs),linestyle=:solid)
lines!(xs,cheb_approx_der_1,linestyle=:dot)
lines!(xs,cheb_approx_der_2,linestyle=:dash)
save("Chebyshev_der_10.png", figure)

CairoMakie.Screen{IMAGE}
